# Imports

In [ ]:
from IPython.display import Image
Image(filename="assets/raalabs.jpg")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
import seaborn as sns
warnings.filterwarnings("ignore")

# Read Data

In [ ]:
churn_df = pd.read_csv('data/customer_churn.csv')
churn_df.head()

# Feature engineering
## One Hot Encoding

In [ ]:
# Binarize area codes
churn_df['Area Code'] = churn_df['Area Code'].apply(str)
pd.get_dummies(churn_df['Area Code']).head()

# Cleaning
## Transform true/false yes/no text into numerics

In [ ]:
churn_df['State'].value_counts()[0:10]

In [ ]:
# fix the outcome
churn_df['Churn?'] = np.where(churn_df['Churn?'] == 'True.', 1, 0)
churn_df["Int'l Plan"] = np.where(churn_df["Int'l Plan"] == 'yes', 1, 0)
churn_df['VMail Plan'] = np.where(churn_df['VMail Plan'] == 'yes', 1, 0)

In [ ]:
# dummify states
pd.get_dummies(churn_df['State']).head()

In [ ]:
# binarize categorical columns
churn_df = pd.concat([churn_df, pd.get_dummies(churn_df['State'])], axis=1)
churn_df = pd.concat([churn_df, pd.get_dummies(churn_df['Area Code'])], axis=1)

churn_df.head()

In [ ]:
# check for nulls in data and impute if necessary
for feat in list(churn_df):
     if (len(churn_df[feat]) - churn_df[feat].count()) > 0:
         print(feat)
         print(len(churn_df[feat]) - churn_df[feat].count())
         tmp_df.loc[tmp_df[feat].isnull(), feat] = 0

In [ ]:
list(churn_df)

## Split into Features and Target

In [ ]:
features = [feat for feat in list(churn_df) if feat not in ['State', 'Churn?', 'Phone', 'Area Code']]

In [ ]:
outcome = 'Churn?'

# Create XGBoost Classification prediction model

## Split into test (70%) and train (30%)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(churn_df, 
                                                 churn_df[outcome], 
                                                 test_size=0.3, 
                                                 random_state=42)

## Select evaluation metric (AUC)

In [ ]:
import xgboost as xgb
xgb_params = {
    'max_depth':3, 
    'eta':0.05, 
    'silent':0, 
    'eval_metric':'auc',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective':'binary:logistic',
    'seed' : 0
}

## Cast to matrix (XGBoost requirement)

In [ ]:
dtrain = xgb.DMatrix(X_train[features], y_train, feature_names = features)
dtest = xgb.DMatrix(X_test[features], y_test, feature_names = features)
evals = [(dtrain,'train'),(dtest,'eval')]
xgb_model = xgb.train (params = xgb_params,
              dtrain = dtrain,
              num_boost_round = 2000,
              verbose_eval=50, 
              early_stopping_rounds = 500,
              evals=evals,
              #feval = f1_score_cust,
              maximize = True)

# TODO: Learn about AUC before continuing!

# Plot ROC Curve

In [ ]:
print("Validating...")
check = xgb_model.predict(xgb.DMatrix(X_test[features]), ntree_limit=xgb_model.best_iteration+1)

In [ ]:
from sklearn.metrics import average_precision_score
#area under the precision-recall curve
score = average_precision_score(X_test[outcome].values, check)
print('area under the precision-recall curve: {:.6f}'.format(score))



In [ ]:
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
check2=check.round()
score = precision_score(X_test[outcome].values, check2)
print('precision score: {:.6f}'.format(score))


In [ ]:
score = recall_score(X_test[outcome].values, check2)
print('recall score: {:.6f}'.format(score))

In [ ]:
print("Predict test set... ")
test_prediction = xgb_model.predict(xgb.DMatrix(X_test[features],missing = -99), ntree_limit=xgb_model.best_iteration+1)
score = average_precision_score(X_test[outcome].values, test_prediction)

print('area under the precision-recall curve test set: {:.6f}'.format(score))

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(X_test[outcome].values, check)
roc_auc = auc(fpr, tpr)
#xgb.plot_importance(gbm)
#plt.show()
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

# Let's not stop here, let's get some actionable insight out of this!

# Plot feature importance

In [ ]:
fig, ax = plt.subplots(figsize=(6,9))
xgb.plot_importance(xgb_model,  height=0.8, ax=ax)
plt.show()

# Spit feature importance into a dataframe

In [ ]:
# get dataframe version of important feature for model 
xgb_fea_imp=pd.DataFrame(list(xgb_model.get_fscore().items()),
columns=['feature','importance']).sort_values('importance', ascending=False)
xgb_fea_imp.head(10)

# Creating top/bottom percentiles to determine under/over use

### Is customer under 75 percentile = not using service enough?

In [ ]:
churn_df['Day Mins'].quantile(0.25)

In [ ]:
churn_df['Day Mins'].quantile(0.75)

## Plotting number of customers who are going to churn

In [ ]:
pred_churn = xgb_model.predict(dtest)
plt.plot(sorted(pred_churn))
plt.grid()

## Loop thorugh each numerical feature and get 25 and 75 percentile for each customer with high risk of churning

In [ ]:
# get all numerical features
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_features = list(X_test.head().select_dtypes(include=numerics))
features_to_ignore = ['Account Length', 'Area Code','Churn?', 'Will_Churn']
numeric_features = [nf for nf in numeric_features if nf not in features_to_ignore]

row_counter = 0
X_test['Will_Churn'] = pred_churn
new_df = []
for index, row in X_test.iterrows():
    if row['Will_Churn'] > 0.8:
        row_counter += 1
        new_df.append(row[list(churn_df)])
        for feat in numeric_features:
            # only consider high prob churns
            if row[feat] < X_test[feat].quantile(0.25):
                print('(ID:', row_counter, ')', feat,  ' is < than 25 percentile')
            if row[feat] > X_test[feat].quantile(0.75):
                print('(ID:', row_counter, ')', feat,  ' is > than 75 percentile')


new_df[0]

## Inject one "high risk" customer into a cluster of "low risk" customers and see how it fits.
#### Create a new dataframe and inject high risk customer to the top

In [ ]:
# get all known not to churn
not_churn = X_train[X_train['Churn?']==False].copy()

find_closet_df = []

# add row to find insights
find_closet_df.append(new_df[0])

for index, row in not_churn.iterrows():
    find_closet_df.append(row[list(churn_df)])
    
find_closet_df = pd.DataFrame(find_closet_df)
find_closet_df['ID'] = [idx for idx in range(1,len(find_closet_df)+1)]
find_closet_df.head()

# Run KMeans unsupervised model

<img src="https://www.brandidea.com/images/datascience/kmeansxmeans.jpg" />

# Find Closest Clusters to the Embedded Churn Risk
#### Create 20 clusters and show me the closest customers to starting customer

In [ ]:
from sklearn.cluster import KMeans 
num_clusters = 20
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(find_closet_df[features])
labels = kmeans.labels_
find_closet_df['clusters'] = labels
find_closet_df.head()

# We compare the row with high-probability of churn against non-churns

We find 13 rows of non-churn resembling row 0 with the high-probability of churn, thus we recommend offering day-time credits to this customer

In [ ]:
find_closet_df[find_closet_df['clusters']==6][features]

In [ ]:
find_closet_df.head()

## Get better understanding by plotting into a scatter plot

In [ ]:
def risk_compare(cluster_df, cluster_number, var1, var2):
    mydat = find_closet_df.copy()
    mydat = mydat[mydat['clusters'] == cluster_number]
    mydat = mydat[[var1, var2, 'clusters']]
    # differentiate high-risk churn customer
    mydat.iat[0, 2] = 0

    sns.lmplot(var1, var2, data=mydat,
               fit_reg=False, hue="clusters", 
               scatter_kws={"marker": "D", "s": 100})

    plt.xlabel(var1)
    plt.ylabel(var2)
    plt.show()

### Pass cluster number into function and what columns we want to cluster on

In [ ]:
risk_compare(find_closet_df.copy(), 6, 'Night Mins', 'Night Calls')

In [ ]:
risk_compare(find_closet_df.copy(), 6, 'Day Mins', 'Eve Mins')